In [1]:
import pickle
import wandb
import threading
import torch
import sys
import os
import gc
import contextlib
import transformers

from pathlib import Path

from torch.utils.data import DataLoader
from vllm import LLM, SamplingParams

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from power_monitoring.monitor import HWMonitor

from utils.gpu_management import reset_vllm_gpu_environment

display(f"CUDA available: {torch.cuda.is_available()}")
display(f"cuDNN version: {torch.backends.cudnn.version()}")

os.environ["TOKENIZERS_PARALLELISM"] = "false"

'CUDA available: True'

'cuDNN version: 90100'

In [2]:
dataset = load_dataset('wmt14', 'de-en', split='train')
display(dataset[0])

{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode',
  'en': 'Resumption of the session'}}

In [3]:
with open(f"{Path.home()}/.cache/huggingface/token", "r") as f:
    HF_TOKEN = f.read()
    f.close()

MODELS = ["meta-llama/Llama-3.2-3B"] # , "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16"
SYSTEM_PROMPT = "Translate the following text from German to English. Make sure not to change the meaning:"
MAX_SEQ_LEN = 8192

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MINIBATCH_SIZE = 8
NUM_SAMPLES = 2
SAMPLING_PARAMS = SamplingParams(temperature=0.8, top_p=0.95)

NUM_GPUS = torch.cuda.device_count()

In [4]:
# Dataset processing

def add_instruction(sentence_pair, instruction: str = None):
    sentence_pair["input"] = f"{instruction}\n\n\n{sentence_pair['translation']['de']}"
    sentence_pair["target"] = sentence_pair['translation']['en']
    return sentence_pair

dataset = dataset.shuffle().select(range(NUM_SAMPLES))
dataset = dataset.map(lambda sentence_pair: add_instruction(sentence_pair, SYSTEM_PROMPT), remove_columns=['translation'])
display(dataset[0])

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

{'input': 'Translate the following text from German to English. Make sure not to change the meaning:\n\n\nIch war einmal Gesundheitsminister in einem Mitgliedstaat, und ich kann Ihnen sagen, dass die Zahl der Toten, aber auch der Unfälle, in deren Folge junge Menschen gelähmt wurden, mich bis ans Ende meiner Tage verfolgen wird.',
 'target': 'I have been Minister for Health in a Member State and I can tell you that the number of deaths, and also the number of accidents that leave young people quadriplegic will affect me for the rest of my life.'}

In [5]:
# display(dataset["input"])
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", token=HF_TOKEN)

In [6]:
def embed_samples(sentence_pair, tokenizer): 
    sentence_pair["prompt"] = tokenizer.apply_chat_template(sentence_pair["input"], add_generation_prompt=True, tokenize=False)
    return sentence_pair


for model_name in MODELS: 
    # reset_vllm_gpu_environment()
    
    pipe = transformers.pipeline(
        "text-generation", 
        model=model_name, 
        # model_kwargs={"torch_dtype": torch.bfloat16}, 
        device_map="auto"
    )

    pipe("Hi, who are you?")
    
    

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
